In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'navnode_fetch',
                        'payload': '{"jsn":{"tab":"extras","groupname":"' + input_.loc[a, 'Category'] + '","nodetype":"groupname"}}',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(json_data['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//input[contains(@id, "jsn")]/@value')
                
                # = = = = = = = = = = = = = = =
                
                list_subcategory = [json.loads(part)['subgroupname'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Category_No.': int(input_.loc[a, 'Category_No.']),
                                        'Category': input_.loc[a, 'Category'],
                                        'Subcategory_No.': [i+1 for i in range(len(list_part))],
                                        'Subcategory': list_subcategory})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Category_No.': int(input_.loc[a, 'Category_No.']),
                                     'Category': input_.loc[a, 'Category']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Category'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Category_No.', 'Subcategory_No.'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./2.subcategory-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Category_No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./subcategory_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：26

[ok] - Body & Lamp Assembly
[尝试次数：1] - [剩余数量：6] - [当前时间：18:07:54]

[ok] - Drivetrain
[尝试次数：1] - [剩余数量：5] - [当前时间：18:07:54]

[ok] - Electrical-Switch & Relay
[尝试次数：1] - [剩余数量：4] - [当前时间：18:07:54]

[ok] - Literature
[尝试次数：1] - [剩余数量：3] - [当前时间：18:07:54]

[ok] - Electrical-Bulb & Socket
[尝试次数：1] - [剩余数量：2] - [当前时间：18:07:54]

[ok] - Cooling System
[尝试次数：1] - [剩余数量：1] - [当前时间：18:07:54]

[ok] - Hoses/Lines & Clamps
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:54]

[ok] - Belt Drive
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:54]

[ok] - Exhaust & Emission
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:54]

[ok] - Electrical
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:54]

[ok] - Ignition
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:54]

[ok] - Hardware
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:54]

[ok] - Garage Equipment
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:54]

[ok] - Suspension
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:55]

[ok] - Engine
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:55]

[ok] - Heat & Air Conditioning
[尝试次数：1] - [剩余数量：0] - [当前时间：18:07:55]

[ok] - T